# Automated Machine Learning with Auto-Keras

## Imports and Version Selection

In [0]:
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Activation, Input, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

!pip install --upgrade deeplearning2020
from deeplearning2020 import helpers

# jupyters magic command
%matplotlib inline

Requirement already up-to-date: deeplearning2020 in /usr/local/lib/python3.6/dist-packages (0.4.18)


## Loading and Preprocessing

In [0]:
# download the dataset with labels and with information about the data
data, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

# print the most important information
dataset_size = info.splits['train'].num_examples
print('dataset size: ', dataset_size)
class_names = info.features['label'].names
print('class names: ', class_names)
n_classes = info.features['label'].num_classes
print('number of classes: ', n_classes)

# resize the images to a uniform size
def preprocess(image, label):
  resized_image = tf.image.resize(image, [224, 224])
  resized_image /= 255

  # one-hot encode the labels, because autokeras needs this form
  label = tf.one_hot(label, n_classes)
  return resized_image, label

batch_size = 32

try:
  train_data = tfds.load('tf_flowers', split="train[:80%]", as_supervised=True)
  test_data = tfds.load('tf_flowers', split="train[80%:100%]", as_supervised=True)
  train_data = train_data.shuffle(1000).map(preprocess)
  test_data = test_data.map(preprocess)
except(Exception):
  # split the data into train and test data with a 8:2 ratio
  train_split, test_split = tfds.Split.TRAIN.subsplit([8, 2])
  train_data = tfds.load('tf_flowers', split=train_split, as_supervised=True)
  test_data = tfds.load('tf_flowers', split=test_split, as_supervised=True)
  train_data = train_data.shuffle(1000).map(preprocess)
  test_data = test_data.map(preprocess)

print(train_data)
# convert tf.dataset to numpy array
# X, y, X_test, y_test = helpers.dataset_to_ndarray(train_data, test_data)

# X = X/255.0
# X_test = X_test/255.0

dataset size:  3670
class names:  ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
number of classes:  5
<DatasetV1Adapter shapes: ((224, 224, 3), (5,)), types: (tf.float32, tf.float32)>


## Additional necessary import

In [0]:
!pip install autokeras
import autokeras as ak

## Definition and Training

In [0]:
clf = ak.ImageClassifier(max_trials=10)

In [0]:
# Auto-Keras often has a lot of bugs, so don't be surprised in case it doesn't work as expected
clf.fit(train_data)

## Validierung

In [0]:
clf.evaluate(test_data)